In [1]:
from tool import *
import pyautogui
import sys

# general options
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

## PO DB 업데이트

In [20]:
filepath = 'D:/Downloads/' + [file for file in os.listdir('D:/Downloads/') if 'poshThru' in file][-1] # 파일이름에 poshThru가 들어가는 파일 중에 가장 최신 파일을 선택함
df = pd.read_excel(filepath, sheet_name='DataSet').iloc[:, :48]

if df['Model'][0].split('.')[0] == 'PH30N':
    vendor = 'Wanlida'
else:
    vendor = df['Model'].apply(lambda x:x.split('-')[0]).replace(srt_model).replace(vendor_find)[0]

if vendor not in ['Quanta', 'Pegatron', 'Wanlida', 'Wingtech']:
    sys.exit()

df = df[['Model', 'PO No.', 'Ship To', 'Shipping', 'Cancel', 'PO', 'OQC Report', 'OQC Date', 'OQC Result', 'Ship', 'Issued Date', 'RSD','Ship Date', 'BL No', 'BL Status', 'Method', 'Price Term', 'Unit Price', 'Currency', 'Payment Term', 'PO1 No', 'SO No.', 'Final Destination']]
df.loc[df['PO1 No'].notnull(), 'PO1 No'] = df.loc[df['PO1 No'].notnull(),'PO1 No'].astype('int').astype('str')
df.loc[df['SO No.'].notnull(), 'SO No.'] = df.loc[df['SO No.'].notnull(), 'SO No.'].astype('int').astype('str')
df[['OQC Date', 'PO1 No', 'SO No.', 'Final Destination']] = df[['OQC Date', 'PO1 No', 'SO No.', 'Final Destination']].fillna('-')

df['Ship'].fillna(0, inplace=True)
df['Ship'] = df['Ship'].astype('int')
df['Issued Date'] = pd.to_datetime(df['Issued Date'])
df['PO Week'] = df['Issued Date'].dt.isocalendar().week.apply(lambda x:'{0:02d}'.format(x))
df['PO Month'] = df['Issued Date'].apply(get_month_from_date)
df['PO Year'] = df['Issued Date'].dt.isocalendar().year
df['RSD'] = pd.to_datetime(df['RSD']) # PO의 RSD를 Datetime 형식으로 변경
df['RSD Week'] = df['RSD'].dt.isocalendar().week.apply(lambda x:'{0:02d}'.format(x))
df['RSD Month'] = df['RSD'].apply(get_month_from_date)
df['RSD Year'] = df['RSD'].dt.isocalendar().year
df['RSD Week Year'] = df['RSD'].apply(getfirstdate_year)
df['RSD Week Month'] = df['RSD'].apply(getfirstdate_month)
df['RSD Week Day'] = df['RSD'].apply(getfirstdate_day)
df['RSD Week Name'] = df['RSD Week Year'].astype(str) + '-' + df['RSD Week Month'].astype(str) + '-' + df['RSD Week Day'].astype(str) + '(W' + df['RSD Week'].astype(str) +')'
df.rename(columns={'Model':'Mapping Model.Suffix'}, inplace=True)

if vendor != 'Wanlida':
    df['Model'] = df['Mapping Model.Suffix'].apply(lambda x: x.split('-')[0])
    df['Series'] = df['Model'].replace(srt_model)

df['Ship To'] = df['Ship To'].replace(site_name_adjust_map)
df['Country'] = df['Ship To'].replace(site_map)
df['Region'] = df['Country'].replace(country_map)

if vendor != 'Wanlida':
    df1 = df.groupby(['PO No.', 'Cancel', 'Issued Date', 'PO Year', 'PO Month', 'PO Week', 'Region', 'Country', 'Series', 'Model',
                        'Mapping Model.Suffix', 'Ship To', 'RSD', 'RSD Year', 'RSD Month', 'RSD Week', 'RSD Week Name', 'Method',
                        'Price Term', 'Unit Price', 'Currency', 'Payment Term', 'PO1 No', 'SO No.',
                        'Final Destination'])[['PO']].sum().reset_index().set_index('PO No.')
else:
    df1 = df.groupby(['PO No.', 'Cancel', 'Issued Date', 'PO Year', 'PO Month', 'PO Week', 'Region', 'Country',
                    'Mapping Model.Suffix', 'Ship To', 'RSD', 'RSD Year', 'RSD Month', 'RSD Week', 'RSD Week Name', 'Method',
                    'Price Term', 'Unit Price', 'Currency', 'Payment Term', 'PO1 No', 'SO No.',
                    'Final Destination'])[['PO']].sum().reset_index().set_index('PO No.')  

with open(f'D:/Data/{vendor} PO DB.bin', 'rb') as f:
    PO_df = pickle.load(f)

In [22]:
PO_df1 = PO_df.copy()

In [23]:
prev_po = PO_df.shape[0]

In [24]:
for i in df1.index:
    PO_df.loc[i, :] = df1.loc[i, :]

In [25]:
updated_po = PO_df.shape[0]

In [26]:
with open(f'D:/Data/{vendor} PO DB.bin', 'wb') as f:
    pickle.dump(PO_df, f)
# DB backup 폴더에도 저장
with open(f'D:/Data/DB backup/{vendor} PO DB.bin', 'wb') as f:
    pickle.dump(PO_df, f)

## Shipment result DB 업데이트

In [27]:
with open(f'D:/Data/{vendor} shipment result DB.bin', 'rb') as f:
     SR_df = pickle.load(f)

df = df[df['Ship Date'].notnull()]
df['Ship Date'] = pd.to_datetime(df['Ship Date'])
df['Ship Week'] = df['Ship Date'].dt.isocalendar().week.apply(lambda x:'{0:02d}'.format(x))
df['Ship Month'] = df['Ship Date'].apply(get_month_from_date)
df['Ship Year'] = df['Ship Date'].dt.isocalendar().year
df['Ship Week month'] = df['Ship Date'].apply(getfirstdate_month)
df['Ship Week day'] = df['Ship Date'].apply(getfirstdate_day)
df['Ship Week year'] = df['Ship Date'].apply(getfirstdate_year)
df['Ship Week year'] = df['Ship Week year'].astype(str)
df['Ship Week month'] = df['Ship Week month'].astype(str)
df['Ship Week day'] = df['Ship Week day'].astype(str)
df['Ship Week'] = df['Ship Week'].astype(str)
df['Week Name'] = df['Ship Week year'] + '-' + df['Ship Week month'] + '-' + df['Ship Week day'] + '(W' + df['Ship Week'] + ')'
df = df.set_index(['PO No.', 'BL No'])

In [28]:
SR_df1 = SR_df.copy()

In [29]:
prev_sr = SR_df.shape[0]

In [30]:
for i in df.index:
    SR_df.loc[i, :] = df.loc[i, :]

In [31]:
updated_sr = SR_df.shape[0]

In [32]:
# PO 대비 선적수량이 더 많은 경우가 없는지 check
with open(f'D:/Data/{vendor} PO DB.bin', 'rb') as f:
    PO = pickle.load(f)
with open(f'D:/Data/{vendor} shipment result DB.bin', 'rb') as f:
    SR = pickle.load(f)

PO = PO[PO['Cancel']=='N']
df = pd.concat([PO['PO'], SR.groupby('PO No.')['Ship'].sum()], axis=1)
df.fillna(0, inplace=True)
df['Open'] = df['PO'] - df['Ship']
df = df[df['Open'] < 0]

In [33]:
if df.shape[0] != 0:
    print(df)
    sys.exit()

In [34]:
with open(f'D:/Data/{vendor} shipment result DB.bin', 'wb') as f:
    pickle.dump(SR_df, f)

In [35]:
with open(f'D:/Data/DB backup/{vendor} shipment result DB.bin', 'wb') as f:
    pickle.dump(SR_df, f)
print(f"{vendor}의 DB를 업데이트 하였습니다. PO는 {updated_po - prev_po} 건, 선적은 {updated_sr - prev_sr} 건 추가 되었습니다. ")
print(filepath)

Wingtech의 DB를 업데이트 하였습니다. PO는 3 건, 선적은 7 건 추가 되었습니다. 
D:/Downloads/poshThru_20230904124031.xls


In [36]:
PO_df.drop(index=PO_df1.index).pivot_table('PO', index=['PO No.', 'Mapping Model.Suffix',  'Country'], columns='RSD Week Name', aggfunc=sum) # 추가된 PO 리스트

,,RSD Week Name,2023-08-28(W35),2023-09-18(W38),2023-09-25(W39)
PO No.,Mapping Model.Suffix,Country,,,
ZPCZ23005288,10A30Q-L.AQ1TL,Korea,NaN,300.0,NaN
ZPCZ23005449,10A30Q-L.AQ12ML,Korea,30.0,NaN,NaN
ZPCZ23005457,10A30Q-L.AQ1TL,Korea,NaN,NaN,300.0


In [37]:
SR_df.drop(index=SR_df1.index).pivot_table('Ship', index=['Country', 'BL No', 'Mapping Model.Suffix', 'PO No.'], columns=['Week Name', 'Ship Date'], aggfunc='sum').fillna(0) # 추가된 선적실적

Week Name                                             2023-08-07(W32)  \
Ship Date                                                  2023-08-09   
Country BL No       Mapping Model.Suffix PO No.                         
Korea   BAA23080102 10A30Q-L.AQ14K       ZPCZ23004071           440.0   
        BAA23080103 10A30Q-L.AQ28K       ZPCZ23004070           120.0   
        BAA23080104 10A30Q-L.AQ28ML      ZPCZ23004073           140.0   
        BAA23080105 10A30Q-L.AQ12ML      ZPCZ23004308             0.0   
        BAA23080106 10A30Q-L.AQ12ML      ZPCZ23004308             0.0   
        BAA23080107 10A30Q-L.AQ48KN      ZPCZ23004072             0.0   
        BAA23080385 10A30Q-L.AQ1TL       ZPCZ23004578             0.0   

Week Name                                                         \
Ship Date                                             2023-08-10   
Country BL No       Mapping Model.Suffix PO No.                    
Korea   BAA23080102 10A30Q-L.AQ14K       ZPCZ23004071        0.0   
        BAA23080103 10A30Q-L.AQ28K       ZPCZ23004070        0.0   
        BAA23080104 10A30Q-L.AQ28ML      ZPCZ23004073        0.0   
        BAA23080105 10A30Q-L.AQ12ML      ZPCZ23004308     1440.0   
        BAA23080106 10A30Q-L.AQ12ML      ZPCZ23004308        0.0   
        BAA23080107 10A30Q-L.AQ48KN      ZPCZ23004072        0.0   
        BAA23080385 10A30Q-L.AQ1TL       ZPCZ23004578        0.0   

Week Name                                             2023-08-14(W33)  \
Ship Date                                                  2023-08-18   
Country BL No       Mapping Model.Suffix PO No.                         
Korea   BAA23080102 10A30Q-L.AQ14K       ZPCZ23004071             0.0   
        BAA23080103 10A30Q-L.AQ28K       ZPCZ23004070             0.0   
        BAA23080104 10A30Q-L.AQ28ML      ZPCZ23004073             0.0   
        BAA23080105 10A30Q-L.AQ12ML      ZPCZ23004308             0.0   
        BAA23080106 10A30Q-L.AQ12ML      ZPCZ23004308           160.0   
        BAA23080107 10A30Q-L.AQ48KN      ZPCZ23004072             0.0   
        BAA23080385 10A30Q-L.AQ1TL       ZPCZ23004578             0.0   

Week Name                                             2023-08-28(W35)  
Ship Date                                                  2023-08-31  
Country BL No       Mapping Model.Suffix PO No.                        
Korea   BAA23080102 10A30Q-L.AQ14K       ZPCZ23004071             0.0  
        BAA23080103 10A30Q-L.AQ28K       ZPCZ23004070             0.0  
        BAA23080104 10A30Q-L.AQ28ML      ZPCZ23004073             0.0  
        BAA23080105 10A30Q-L.AQ12ML      ZPCZ23004308             0.0  
        BAA23080106 10A30Q-L.AQ12ML      ZPCZ23004308             0.0  
        BAA23080107 10A30Q-L.AQ48KN      ZPCZ23004072            40.0  
        BAA23080385 10A30Q-L.AQ1TL       ZPCZ23004578           440.0